[See docs/DISCLAIMER_SNIPPET.md](../../../docs/DISCLAIMER_SNIPPET.md)

# Alpha-Factory Finance Demo 📈

This notebook launches a **single-node Alpha-Factory** container, activates
the *BTC/GLD momentum* strategy, then queries the FinanceAgent for **positions**
and **P&L**.  It runs **online or offline** – if `OPENAI_API_KEY` is missing the
container automatically uses a local Φ-2 model.

It uses a simulated exchange by default and should not be used with real funds.


In [ ]:
# 0 · Environment setup
import shutil, subprocess
if not shutil.which('docker'):
    print('⏳ Installing Docker...')
    subprocess.run(['apt-get','update'], check=True)
    subprocess.run(['apt-get','install','-y','docker.io'], check=True)
    subprocess.run(['service','docker','start'], check=True)
else:
    print('✅ Docker already installed')


## 1 · Parameters
Adjust `STRATEGY` for your trading pair and modify `PORT_API` if 8000 is occupied. Set `TRACE_WS_PORT` if 8088 is taken.

In [ ]:
IMG = "ghcr.io/montrealai/alphafactory_pro:cpu-slim-latest"
PORT_API = 8000
TRACE_WS_PORT = 8088
CONTAINER = "af_nb_demo"
STRATEGY = "btc_gld"  # change to your own


## 2 · Start container

In [ ]:
import subprocess, time, requests, socket
print('⏳ Pulling image (skip if cached)…')
subprocess.run(['docker', 'pull', IMG], check=True)

def port_free(port:int)->bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) != 0
if not port_free(PORT_API):
    raise RuntimeError(f'Port {PORT_API} is already in use')
if not port_free(TRACE_WS_PORT):
    raise RuntimeError(f'Port {TRACE_WS_PORT} is already in use')

print('🚀 Starting Alpha‑Factory …')
cid = subprocess.check_output(['docker','run','-d','--rm','--name',CONTAINER,
    '-p', f'{PORT_API}:8000', '-p', f'{TRACE_WS_PORT}:{TRACE_WS_PORT}',
    '-e', f'FINANCE_STRATEGY={STRATEGY}', '-e', f'TRACE_WS_PORT={TRACE_WS_PORT}', IMG], text=True).strip()

for _ in range(90):
    try:
        requests.get(f'http://localhost:{PORT_API}/health', timeout=1)
        break
    except Exception:
        time.sleep(1)
else:
    subprocess.run(['docker','logs',cid])
    raise RuntimeError('API did not start in 90 s')
print(f'✅ API ready at http://localhost:{PORT_API}')


## 3 · Fetch positions & P&L

In [ ]:
import pandas as pd, requests, IPython.display as disp
base = f"http://localhost:{PORT_API}/api/finance"
positions = requests.get(base + "/positions").json()
pnl = requests.get(base + "/pnl").json()

disp.display(pd.json_normalize(positions).style.set_caption("Current Positions"))
disp.display(pd.json_normalize(pnl).style.set_caption("P&L (USD)"))


## 4a · Quick SDK demo
Call the FinanceAgent via the OpenAI Agents SDK (falls back to REST).

In [ ]:
try:
    from openai.agents import AgentRuntime
    rt = AgentRuntime(base_url=f'http://localhost:{PORT_API}', api_key=None)
    fin = rt.get_agent('FinanceAgent')
    print(fin.alpha_signals())
except Exception as e:
    import requests, json
    print('SDK unavailable, using REST → positions:')
    print(json.dumps(requests.get(base + '/positions').json(), indent=2))


## 4 · Explore the trace‑graph ✨
Open [http://localhost:{TRACE_WS_PORT}](http://localhost:{TRACE_WS_PORT}) in your browser to watch
the planner emit decisions and tool‑calls in real time.

In [ ]:
from IPython.display import IFrame, display
display(IFrame(f'http://localhost:{TRACE_WS_PORT}', width='100%', height=500))


## 5 · Shutdown

In [ ]:
import subprocess
subprocess.run(['docker','stop',CONTAINER], check=False)
print('🛑 Container stopped.')
